In [4]:
pip install torch torchvision torchaudio transformers sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 28.5 MB/s  0:00:12m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 39.1 MB/s  0:00:08m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 41.1 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 104.4 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 51.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 34.8 MB/s  0:00:10m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 97.4 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 82.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 104.8 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 99.3 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 65.4 MB/s  0:00:04m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [43]:
import os
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModel
from google.cloud import aiplatform
from google.cloud import aiplatform_v1

In [44]:
PROJECT_ID = "629242692180"  
REGION = "asia-southeast1"     

# Set variables for the current deployed index.
API_ENDPOINT="1148065613.asia-southeast1-629242692180.vdb.vertexai.goog"
INDEX_ENDPOINT="projects/629242692180/locations/asia-southeast1/indexEndpoints/43567048738996224"
DEPLOYED_INDEX_ID="equipment_profile_1765176812403"
QUERY_IMAGE_PATH = "datatest/AI1/20251112_112304(0).jpg"
NUM_NEIGHBORS = 5

In [45]:
client_options = {
  "api_endpoint": API_ENDPOINT
}
vector_search_client = aiplatform_v1.MatchServiceClient(
  client_options=client_options,
)

In [46]:
try:
    processor = AutoProcessor.from_pretrained("google/siglip-base-patch16-224")
    model = AutoModel.from_pretrained("google/siglip-base-patch16-224")
    model.eval()
    print("SigLIP model loaded successfully.")
except Exception as e:
    print(f"Error loading SigLIP model: {e}")
    exit()

SigLIP model loaded successfully.


In [47]:
def create_query_embedding(image_path: str) -> list:
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Query image not found at: {image_path}")

    image = Image.open(image_path).convert("RGB")
    
    # Pre-processing
    inputs = processor(images=image, return_tensors="pt")
    
    # Inference
    with torch.no_grad():
        outputs = model.get_image_features(**inputs)
        query_vector = outputs / outputs.norm(p=2, dim=-1, keepdim=True)
    
    return query_vector.squeeze(0).tolist()

In [48]:
def vector_search(query_image_path: str, index_endpoint: str, deploy_index_id: str, num_neighbors: int):
    try:
        query_embedding_vector = create_query_embedding(query_image_path)
        
        datapoint = aiplatform_v1.IndexDatapoint(
          feature_vector=query_embedding_vector
        )

        query = aiplatform_v1.FindNeighborsRequest.Query(
          datapoint=datapoint,

          # The number of nearest neighbors to be retrieved
          neighbor_count=num_neighbors
        )

        # filter_condition = [
        #     {"namespace": "metadata.label_class", "allow_list": ["AI1"]} # use namespace 'metadata.label_class'
        # ]
        filter_condition = None
        
        try:
            request = aiplatform_v1.FindNeighborsRequest(
              index_endpoint=index_endpoint,
              deployed_index_id=deploy_index_id,
              # Request can have multiple queries
              queries=[query],
              return_full_datapoint=False,
            )

            # Execute the request
            response = vector_search_client.find_neighbors(request)

            # Handle the response
            print(response)
        except Exception as e:
            print(f"ERROR: {e}")
            return
        
    except Exception as e:
        print(f"Error creating embedding: {e}")
        return
    return 

In [49]:
res = vector_search(QUERY_IMAGE_PATH, INDEX_ENDPOINT, DEPLOYED_INDEX_ID, NUM_NEIGHBORS)

nearest_neighbors {
  neighbors {
    datapoint {
      datapoint_id: "equipment_train_AI1_20251112_112303.jpg"
      crowding_tag {
        crowding_attribute: "0"
      }
    }
    distance: 0.98945379257202148
  }
  neighbors {
    datapoint {
      datapoint_id: "equipment_train_AI1_20251112_112304.jpg"
      crowding_tag {
        crowding_attribute: "0"
      }
    }
    distance: 0.98305577039718628
  }
  neighbors {
    datapoint {
      datapoint_id: "equipment_train_AI1_20251112_112304(1).jpg"
      crowding_tag {
        crowding_attribute: "0"
      }
    }
    distance: 0.98234450817108154
  }
  neighbors {
    datapoint {
      datapoint_id: "equipment_train_AI1_20251112_112305(2).jpg"
      crowding_tag {
        crowding_attribute: "0"
      }
    }
    distance: 0.97134977579116821
  }
  neighbors {
    datapoint {
      datapoint_id: "equipment_train_AI1_20251112_112305.jpg"
      crowding_tag {
        crowding_attribute: "0"
      }
    }
    distance: 0.96499359607